In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm.notebook import tqdm
from multiprocess import Pool
from tensorflow.keras.utils import Sequence
import _pickle as pickle
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

def clean_name(x):
    x = x.replace('/', '_')
    for j in '.?!:,':
        x = x.replace(j, '')
    x = x.replace(' ', '_')
    return x


from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils import np_utils

from transformers import TFBertModel, TFRobertaModel

from transformers import BertTokenizer, RobertaTokenizer

Using TensorFlow backend.


## add negative keywords

In [2]:
dico_prompt  = load('dico prompt revuze')

topic_list = list(dico_prompt.keys())

bert_model = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(bert_model, do_lower_case = True)

max_len = 128

In [ ]:
# df_prompt = pd.read_excel('topic prompt.xlsx')

# dico_prompt = {}

# for i, elt in df_prompt.iterrows():
#     d_temp = {
#         'Category1' : str(elt['Category 1']),
#         'Topic 1' : str(elt['Topic 1']),
#         'Topic 2' : str(elt['Topic 2']),
#         'Definition' : str(elt['Definition of the topic']),
#         'Questions' : [str(q) for q in elt[['Question1', 'Question2','Question3']].values if not(str(q) == 'nan')],
#         'keywords' : [" ".join(str(k).split(' ')) for k in elt['kwd'].split(', ') if not(k == ' ')],
#     }
# #     print([" ".join(str(k).split(' ')) for k in elt['kwd'].split(', ') if not(k == '')])
#     dico_prompt[str(elt['topic'])] = d_temp

# save(dico_prompt, 'dico prompt revuze')

In [ ]:
# data = load('full revuze data')
# data = data[['topic', 'text']]

# sample = data.sample(n = 100000)
# save(sample, 'sample_100k')

In [ ]:
# data = load('full revuze data')
# data = data[['topic', 'text']]
# data.index = data['topic']
# data.index = list(range(data.shape[0]))
# def save_batch(df, name, batch_size):
#     size = df.shape[0]
#     df1 = df.sample(n = size)
#     n_batch = int(size / batch_size) + 1
    
#     dir1 = './batch/'+clean_name(name)
#     print(dir1)
#     try:
#         os.mkdir(dir1)
#     except:
#         1
    
#     for i in tqdm(range(n_batch), leave = False):
#         d = df1.iloc[batch_size*i : batch_size * (i+1)]['text'].values
#         save(d, clean_name(name) +'_'+str(i), dir1[2:])
        
#     return d

# for topic, frame in tqdm(data.groupby('topic')):
#     print(topic)
#     save_batch(frame, topic, 5000)

In [3]:
def build_prompt(t1, t2, t3, q, k):
    return "</s></s> ".join([t1,t2,t3,q,k])

def build_topic_prompt(topic, dico_prompt):
    t = dico_prompt[topic]
    return build_prompt(t['Topic 1'], t['Topic 2'], topic, random.choice(t['Questions']), 
                        ','.join(np.random.choice(t['keywords'], replace = True, size = random.randint(1,5))))

def prompt_sentence(topic, sentence, tokenizer = tokenizer, dico_prompt = dico_prompt, max_len = 64, ):
    s1 = build_topic_prompt(topic, dico_prompt)
    s2 = sentence
    encoded = tokenizer.encode_plus(s1, s2, add_special_tokens = True, max_length = max_len, pad_to_max_length = True)
    return encoded['input_ids'], encoded['attention_mask']

In [4]:
def gen_true_exemple(topic):
    from utils import clean_name
    from utils import load
    import os
    import numpy as np
    import random
    ## Choice of file
    topic_dir = './batch/'+clean_name(topic)
    file_list = os.listdir(topic_dir)
    file_name = random.choice(file_list).split('.')[0]
    file = load(file_name, topic_dir[2:])
    exemple = random.choice(file)
    return exemple

def gen_false_exemple(topic, topic_list = topic_list, gen_true_exemple = gen_true_exemple):  
    import random
    liste = [elt for elt in topic_list if not(elt == topic)]
    false_topic = random.choice(liste)
    exemple = gen_true_exemple(false_topic)
    return exemple

In [5]:
class data_generator(Sequence):
    def __init__(self, topic_list, batch_size = 32, negative_rate = 0.5, n_jobs = 8, max_len = 64):
        self.topic_list = topic_list
        self.batch_size = batch_size
        self.negative_rate = negative_rate
        self.nsize = int(self.batch_size * self.negative_rate)
        self.psize = self.batch_size - self.nsize
        self.p = Pool(n_jobs)
        self.max_len = max_len
        
    def __len__(self):
        return int(10000000)
     #   return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        ## positive sample
        positive_topic = np.random.choice(self.topic_list, replace = True, size = self.psize)
        positive_label = np.zeros(self.psize) + 1
        positive_exemple = self.p.map(gen_true_exemple, positive_topic)
        
        
        ## negative sample
        negative_topic = np.random.choice(self.topic_list, replace = True, size = self.nsize)
        negative_label = np.zeros(self.nsize)
        negative_exemple = self.p.map(gen_false_exemple, negative_topic)
        
        ## Merging
        topic = np.concatenate([positive_topic, negative_topic])
        label = np.concatenate([positive_label, negative_label])
        exemple = np.concatenate([positive_exemple, negative_exemple])
        
        shuffle_id = np.array(range(self.batch_size))
        np.random.shuffle(shuffle_id)
        topic = topic[shuffle_id]
        label = label[shuffle_id]
        exemple = exemple[shuffle_id]
        
        ids = []
        masks = []
        for i in range(self.batch_size):
            j, m = prompt_sentence(topic[i], exemple[i], 
                                   tokenizer = tokenizer, dico_prompt = dico_prompt, max_len = self.max_len)
            ids.append(j)
            masks.append(m)
            
        train = [np.array(ids), np.array(masks)]
        
        return train, np_utils.to_categorical(label)
    
    def close(self):
        self.p.close()

In [6]:
gen = data_generator(topic_list, batch_size = 64, negative_rate = 0.5, n_jobs = 12, max_len = max_len)

In [7]:
%%time
x, y = gen.__getitem__(1)

Wall time: 339 ms


In [8]:
gen.close()

In [9]:
x

[array([[   0,  253,  356, ...,    1,    1,    1],
        [   0,  937,  449, ...,    1,    1,    1],
        [   0, 3568, 1589, ...,    1,    1,    1],
        ...,
        [   0, 2502,    2, ...,    1,    1,    1],
        [   0,  253,  356, ...,    1,    1,    1],
        [   0, 2502,    2, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])]

In [10]:
for elt in x[0]:
    print(tokenizer.decode(elt))
    print('\n')

<s> end look</s></s> smooth/even look</s></s> separation/clumping</s></s> does the product clump when applied?</s></s> gooped,clump,clump,gooped,gooped</s></s> i experienced absolutely no clumps with this mascara, which i would say is very rare in my experience, so if that is a big concern for you then i would absolutely recommend this.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


<s> general kpi</s></s> purchase intent</s></s> gift purposes</s></s> did you get it as a gift?</s></s> present,gift,present</s></s> great gift</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [11]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.

In [6]:
def build_model(max_len, bert_model):
    inputs_ids = Input(shape = (max_len,), dtype = 'int32')
    inputs_mask = Input(shape = (max_len,), dtype = 'int32')
    inputs = [inputs_ids, inputs_mask]
    sentence_encoder = TFRobertaModel.from_pretrained(
                        bert_model, # Use the 12-layer BERT model, with an uncased vocab.  
                        output_attentions = False, # Whether the model returns attentions weights.
                        output_hidden_states = False, # Whether the model returns all hidden-states.
                    )
    encoded = sentence_encoder(inputs_ids, attention_mask = inputs_mask)[0]
    encoded = encoded[:,0,:]
    output = Dense(2, activation='softmax')(encoded)
    model = Model(inputs, output) 
    return model

In [7]:
model = build_model(max_len, bert_model)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 128, 768), ( 124645632   input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 768)]        0           tf_roberta_model[0][0]           
______________________________________________________________________________________________

In [15]:
initial_lr = 3e-5
loss_classif     =  'categorical_crossentropy'# find the right loss for multi-class classification
optimizer        =  Adam(initial_lr, 1e-8) # find the right optimizer
metrics_classif  =  ['accuracy']

model.compile(loss=loss_classif,
                          optimizer=optimizer,
                          metrics=metrics_classif)

In [16]:
topic_train, topic_test, _,_ = train_test_split(topic_list, topic_list, test_size = 0.2)

In [17]:
topic_test

['Ingredients',
 'Finish',
 'Quality',
 'Stickiness',
 'Unisex',
 'Blending',
 'Occasion appropriate',
 'Smell',
 'Skin Coverage',
 'Sillage',
 'Skin tone',
 'Warranty',
 'Overall satisfaction',
 'Residue']

In [18]:
train_gen = data_generator(topic_train, batch_size = 16, negative_rate = 0.7, n_jobs = 12, max_len = max_len)
test_gen = data_generator(topic_test, batch_size = 1024, negative_rate = 0.7, n_jobs = 12, max_len = max_len)

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose = 1,min_delta=0.005,
                              patience=3, min_lr=3e-7)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=1, mode='auto',baseline=None, restore_best_weights=True)

epochs = 10
step_per_epoch = 200

x_test, y_test = test_gen.__getitem__(1)

model.fit(train_gen, validation_data = (x_test, y_test),epochs=epochs,  
          steps_per_epoch= step_per_epoch, callbacks=[early_stop, reduce_lr])

  ...
    to  
  ['...']
Train for 200 steps, validate on 1024 samples
Epoch 1/10
200/200 [==============================] - 93s 466ms/step - loss: 0.6386 - accuracy: 0.6794 - val_loss: 0.6110 - val_accuracy: 0.6992
Epoch 2/10
200/200 [==============================] - 80s 402ms/step - loss: 0.6271 - accuracy: 0.6850 - val_loss: 0.5388 - val_accuracy: 0.7725
Epoch 3/10
200/200 [==============================] - 81s 404ms/step - loss: 0.4450 - accuracy: 0.8200 - val_loss: 0.4027 - val_accuracy: 0.8604
Epoch 4/10
200/200 [==============================] - 81s 406ms/step - loss: 0.2741 - accuracy: 0.9022 - val_loss: 0.3312 - val_accuracy: 0.8916
Epoch 5/10
200/200 [==============================] - 81s 403ms/step - loss: 0.2769 - accuracy: 0.9075 - val_loss: 0.3454 - val_accuracy: 0.8916
Epoch 6/10
200/200 [==============================] - 81s 406ms/step - loss: 0.2250 - accuracy: 0.9262 - val_loss: 0.2856 - val_accuracy: 0.8965
Epoch 7/10
200/200 [==============================] - 80s 4

In [20]:
train_gen.close()
test_gen.close()

In [22]:
model.save_weights('test_topic_robert.h5')

## Testing

In [8]:
model.load_weights('test_topic_robert.h5')

In [9]:
dico_prompt

{'Absorbance': {'Category1': 'Cosmetics,Skincare,',
  'Topic 1': 'Application',
  'Topic 2': 'Blending',
  'Definition': "Does it blend well on the skin and don't sink in the skin and is well absorbed by the skin",
  'Questions': ['Is it correctly absorbed by the skin?',
   'Does it sink correctly in the skin?',
   'Does it blend quickly in the skin?'],
  'keywords': ['blend', 'sink', 'absorb', 'soaked', 'skin', 'absorption']},
 'Acne': {'Category1': 'Cosmetics,Skincare,',
  'Topic 1': 'End Look',
  'Topic 2': 'Conceals imperfection',
  'Definition': 'Does it interract with or provoke acne or breakouts',
  'Questions': ['Does it cover acne?',
   'Are the pimple still visible?',
   'Is it efficace against buttons?'],
  'keywords': ['acne', 'breakout', 'pimple']},
 'Anti-Aging': {'Category1': 'Cosmetics,Skincare,',
  'Topic 1': 'End Look',
  'Topic 2': 'Conceals imperfection',
  'Definition': 'Does it cover fine lines/ wrinkles',
  'Questions': ['Does it make you look younger?',
   'Does

In [10]:
level1 =[]
level2 = []
level3=[]
for topic in dico_prompt:
    level3.append(topic)
    level1.append(dico_prompt[topic]['Topic 1'])
    level2.append(dico_prompt[topic]['Topic 2'])

topics = pd.DataFrame({'level1':level1, 'level2':level2, 'level3':level3})
pd.set_option('display.max_rows', 150)
topics.groupby(['level1', 'level2', 'level3']).sum()

Empty DataFrame
Columns: []
Index: [(Application, Blending, Absorbance), (Application, Blending, Blending), (Application, Ease of application, Ease of use), (Application, Ease of application, Instructions), (Application, Spreading, Spreadability), (Application, Texture, Skin texture), (Customer service, Availability, Discontinued products), (Customer service, Delivery, Shipping), (Customer service, Packaging, Leakage), (Customer service, Packaging, Packaging), (Customer service, Promotion, Promotions), (Customer service, Reliability, Product Defect), (Customer service, Return, Return & refund), (Customer service, Warranty, Warranty), (End Look, Adapted, Age appropriate), (End Look, Adapted, Blend in), (End Look, Adapted, Occasion appropriate), (End Look, Adapted, Seasonality), (End Look, Adapted, Skin tone), (End Look, Adapted, Skin type), (End Look, Cakeing, Residue), (End Look, Color, Color), (End Look, Conceals imperfection, Acne), (End Look, Conceals imperfection, Anti-Aging), (End Look, Conceals imperfection, Dark circles & Puffy eyes), (End Look, Conceals imperfection, Skin pores & Blackheads), (End Look, Coverage, Skin Coverage), (End Look, Finish, Finish), (End Look, Finish, Receiving compliments), (End Look, Finish, Shine), (End Look, Smooth / Even look, Young looking), (End Look, Smooth/Even look, Natural looking), (End Look, Volumizing, Volumizing), (End look, Smooth/Even look, Separation/Clumping), (Feel / Comfort, Comfort, Product Richness), (Feel / Comfort, Comfort, Refreshing), (Feel / Comfort, Discomfort, Eye sensitivity), (Feel / Comfort, Feel Liking, Personal feeling), (Feel / Comfort, Grease / Oil, Oily/Greasy), (Feel / Comfort, Hydration, Hydration), (Feel / Comfort, Smooth, Soft & smooth), (Feel / Comfort, Sticky / Tacky, Stickiness), (General KPI, Eco Friendly, Environmental), (General KPI, Expectation, Meets expectations), (General KPI, Liking / Satisfaction, Overall satisfaction), (General KPI, Liking / Satisfaction, Performance), (General KPI, Liking / Satisfaction, Professional care), (General KPI, Purchase intent, Gift purposes), (General KPI, Purchase intent, Loyalty), (General KPI, Purchase intent, Returning customer), (General KPI, Recomendation, Is it recommended?), (Product, Composition, Ingredients), (Product, Design, Bottle design), (Product, Gender, Gender), (Product, Gender, Unisex), (Product, Price, Price/Value for money), (Product, Quality, Quality), (Product, Size, Portability), (Removal, Removal, Cleansing & Removing), (Smell, Pleasant Smell, Smell), (Smell, Pleasant Smell, Types of fragrances), (Smell, Smell Presence, Notes), (Smell, Smell intensity, Fragrance sensitivity), (Smell, Smell intensity, Fragrance strength), (Smell, Smell intensity, Sillage), (Wear / Long Lasting, Environment resistance, Waterproof), (Wear / Long Lasting, Long Lasting, Long lasting)]

In [11]:
test = load('sample_100k')

In [21]:
sample_size = 2000
sample = test.sample(n = sample_size)['text'].values

In [22]:
%%time

topic1 = 'End look'
topic2 = 'Straightening'
topic3 = 'Straightening'
question = 'does it straighten your hair?'
keywords = 'straight, hair, not curly'

def encode(t1, t2, t3, q, k, sentence):
    prompt = build_prompt(t1, t2, t3, q, k)
    encoded = tokenizer.encode_plus(prompt, sentence, add_special_tokens = True, max_length = max_len, pad_to_max_length = True)
    return encoded['input_ids'], encoded['attention_mask']

input_ids = []
input_masks = []

for elt in tqdm(sample):
    ids, masks = encode(topic1, topic2, topic3, question, keywords, elt)
    input_ids.append(ids)
    input_masks.append(masks)

inputs = [np.array(input_ids), np.array(input_masks)]

pred = model.predict(inputs)[:,1]

indices = np.argsort(pred)


Wall time: 13.4 s


In [16]:
## Top predicted
for i in range(1,11):
    print(sample[indices[-i]])
    print(pred[indices[-i]])

I have naturally curly hair, but I also straighten sometimes.
0.9902194
I have fine but curly hair easier to manage when straightened.
0.9896382
I have naturally curly hair and when I use my usual hair conditioner my hair is way less frizzy.
0.9894462
This is perfect for curly hair.
0.98936355
Great for my curly hair!
0.9892105
I have fine straight hair that still likes to get frizzy.
0.9891053
I have super straight hair so i would spray a little of this on my dry hair, wiggle it around with my hands, and my hair would look and feel thicker.
0.9889635
I have naturally curly hair and it helped keeps that curl!
0.9888614
The best for thick curly hair.
0.98867744
It made my curly, fine hair very soft, shiny and smelling great.
0.988482


In [176]:
## Bottom predicted
for i in range(5):
    print(sample[indices[i]])
    print(pred[indices[i]])

Probably one of my favorites and the packaging is so nice
0.0037312328
Im so in love with this new packaging, smell and possibly improved formula.
0.0037620221
Great colour and coverage, stays put all night!
0.003782531
This also comes in s smaller travel friendly size.
0.0037974594
its easy to use and effective, i don't have to sit there half an hour and play with them.
0.0038216994
